# Example Known Object Labeling

This notebook serves as an example (and usable tool) for labeling objects in the results file as corresponding to a known object. It assumes the user has run KBMOD to produce a results .ecsv and has access to a table of known results.

This notebook uses specific files and parameters from the DEEP reprocessing.

In [ ]:
from astropy.table import Table
import numpy as np

from kbmod.filters.known_object_filters import KnownObjsMatcher
from kbmod.results import Results

We start by loading the results data and known object data. The results data is the ecsv file produced by a KBMOD run and contains information on each trajectory found. The known object table is a given file with information on the location (RA, dec) of each observation at different time steps.

We also extract the required metadata (global WCS and a list of all observation times) from the results.

In [ ]:
# These two files are specific to the UW DEEP reprocessing runs and should be replaced
# by the user's files of interest.
res_file = "/epyc/projects/kbmod/runs/DEEP/results/20190402_A0b_001.results.ecsv"
known_file = "/epyc/projects/kbmod/data/fakes_detections_joined.fits"

in_results = Results.read_table(res_file)
print(f"Loaded a results table with {len(res_file)} entries and columns:\n{in_results.colnames}")

wcs = in_results.wcs
if wcs is None:
    raise ValueError("WCS missing from results file.")

if "mjd_mid" in in_results.table.meta:
    obstimes = np.array(in_results.table.meta["mjd_mid"])
else:
    raise ValueError("Metadata 'mjd_mid' missing from results file.")
print(f"Loaded {len(obstimes)} timestamps.")

known_table = Table.read(known_file)
print(f"\n\nLoaded a known objects table with {len(known_table)} entries and columns:\n{known_table.colnames}")

We use the `KnownObjsMatcher` to determine which of the found results correspond to previously known objects. `KnownObjsMatcher` provides the ability to match by either the number or ratio of observations that are in close proximity to the known object. Here we use a minimum number with reasonable proximity thresholds in space and time.

In [ ]:
min_obs = 10
fake_matcher = KnownObjsMatcher(
    known_table,
    obstimes,
    matcher_name="known_matcher",
    sep_thresh=2.0,  # Obs must be within 2 arcsecs.
    time_thresh_s=600.0,  # Obs must be within 10 minutes.
    name_col="ORBITID",  # For the DEEP-data known objects only.
)

# First create the matches column.
fake_matcher.match(in_results, wcs)

# Second filter the matches.
fake_matcher.match_on_min_obs(in_results, min_obs)

matched_col_name = fake_matcher.match_min_obs_col(min_obs)
print(f"Matches stored in column '{matched_col_name}'")

Iterate over the matched column computing a Boolean of whether there was any match (True if the match list is not empty). Add the resulting list as a new "is_known" column.

In [ ]:
is_known = ~in_results.is_empty_value(matched_col_name)
in_results.table["is_known"] = is_known
matched_count = np.count_nonzero(is_known)

print(f"Found {matched_count} of the {len(in_results)} results matched known objects.")

We could save the resulting joined table using:
```
in_results.write_table(output_filename)
```